In [135]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [144]:
import numpy as np
import json
from aml_storage import Labels, Block, Descriptor
from utils.builder import DescriptorBuilder
import ase.io
from itertools import product
from utils.hamiltonians import fix_pyscf_l1, dense_to_blocks, blocks_to_dense
import matplotlib.pyplot as plt

In [8]:
hams = np.load("data/water-hamiltonian/water_fock.npy", allow_pickle=True)
# MUST CONVERT FROM PYSCF L=1 FORMAT

In [4]:
frames = ase.io.read("data/water-hamiltonian/water_coords_1000.xyz",":")

In [134]:
jorbs = json.load(open('data/water-hamiltonian/orbs_def2_water.json', "r"))
orbs = {}
zdic = {"O" : 8, "H":1}
for k in jorbs:
    orbs[zdic[k]] = jorbs[k]

In [151]:
ham_et = dense_to_blocks(hams, frames, orbs)

In [152]:
dense = blocks_to_dense(ham_et, frames, orbs)

In [153]:
np.linalg.norm(dense[0] - hams[0])

4.0842331876159994e-16

In [31]:
ki, kj = 0, 0
ham_builder = DescriptorBuilder(["a1", "n1", "l1", "a2", "n2", "l2"], ["structure", "atom_i", "atom_j"], ["m1", "m2"], ["hamiltonian"])
zdic = {"O": 8, "H": 1}
for A in range(len(frames)):
    frame = frames[A]
    ham = hams[A]
    ki = 0
    for i, fi in enumerate(frame.symbols):
        ai = zdic[fi]
        for ni, li, mi in orbs[fi]:
            kj = 0
            if mi != -li:
                continue
            for j, fj in enumerate(frame.symbols):
                if i>j:
                    continue
                aj = zdic[fj]
                for nj, lj, mj in orbs[fj]:                
                    if mj != -lj:
                        continue
                    block_idx = (ai, ni, li, aj, nj, lj)
                    if block_idx not in ham_builder.blocks:
                        block = ham_builder.add_block(sparse=block_idx, features=np.asarray([[0]], dtype=np.int32), 
                            components=np.asarray(list(product(range(-li,li+1), range(-lj,lj+1)) ), dtype=np.int32 ) )                                                    
                    else:
                        block = ham_builder.blocks[block_idx]
                    
                    block.add_samples(labels=[(A,i,j)], data=np.asarray(ham[ki:ki+2*li+1, kj:kj+2*lj+1]).reshape((1,-1,1)))

                    kj += 2*lj+1
            ki += 2*li+1
ham_et = ham_builder.build()

In [130]:
ki, kj = 0, 0
ham_builder = DescriptorBuilder(["block_type", "a1", "n1", "l1", "a2", "n2", "l2"], ["structure", "atom_i", "atom_j", "ki_base", "kj_base"], ["m1", "m2"], ["hamiltonian"])
zdic = {"O": 8, "H": 1}
for A in range(len(frames)):
    frame = frames[A]
    ham = hams[A]
    ki = 0
    for i, fi in enumerate(frame.symbols):
        ai = zdic[fi]
        ki_base = ki # pointer at where the i-atom block starts
        for ni, li, mi in orbs[fi]:
            kj = 0
            if mi != -li:
                continue
            for j, fj in enumerate(frame.symbols):
                if i<j: # operate only on the lower-triangular block
                    continue                    
                aj = zdic[fj]
                if i==j:
                    block_type = 0  # diagonal
                elif ai==aj:
                    block_type = 1  # same-species
                else:
                    block_type = 2  # different species
                kj_base = kj # pointer at where the j-atom block starts
                for nj, lj, mj in orbs[fj]:                
                    if mj != -lj:
                        continue
                    block_idx = (block_type, ai, ni, li, aj, nj, lj)
                    if block_idx not in ham_builder.blocks:
                        block = ham_builder.add_block(sparse=block_idx, features=np.asarray([[0]], dtype=np.int32), 
                            components=np.asarray(list(product(range(-li,li+1), range(-lj,lj+1)) ), dtype=np.int32 ) )                                                    
                        if block_type == 1:
                            block_asym = ham_builder.add_block(sparse=(-1,)+block_idx[1:], features=np.asarray([[0]], dtype=np.int32), 
                            components=np.asarray(list(product(range(-li,li+1), range(-lj,lj+1)) ), dtype=np.int32 ) )                                                    
                    else:                        
                        block = ham_builder.blocks[block_idx]
                        if block_type == 1:
                            block_asym = ham_builder.blocks[(-1,)+block_idx[1:]]
                    
                    block_data_ij = np.asarray(ham[ki:ki+2*li+1, kj:kj+2*lj+1])

                    if block_type == 1:
                        kj_offset = kj-kj_base
                        ki_offset = ki-ki_base
                        block_data_ji = np.asarray(ham[kj_base+ki_offset:kj_base+ki_offset+2*li+1, ki_base+kj_offset:ki_base+kj_offset+2*lj+1])                        
                        block.add_samples(labels=[(A,i,j, ki_base, kj_base)], data=(block_data_ij+block_data_ji).reshape((1,-1,1))/np.sqrt(2) )
                        block_asym.add_samples(labels=[(A,i,j,ki_base, kj_base)], data=(block_data_ij-block_data_ji).reshape((1,-1,1))/np.sqrt(2) )
                    else:
                        block.add_samples(labels=[(A,i,j,ki_base, kj_base)], data=block_data_ij.reshape((1,-1,1)))                    
                    kj += 2*lj+1
            ki += 2*li+1
ham_et = ham_builder.build()

In [131]:
dense = []
zdic = {8:"O", 1:"H"}
for f in frames:
    norbs = 0
    for ai in f.symbols:        
        norbs += len(orbs[ai])
    ham = np.zeros((norbs, norbs), dtype=np.float64)
    dense.append(ham)

for idx, block in ham_et:
    cur_A = -1
    block_type, ai, ni, li, aj, nj, lj = tuple(idx)
    fi = zdic[ai]
    fj = zdic[aj]
    ki_offset = 0
    for no, lo, mo in orbs[fi]:        
        if no == ni and lo == li:
            break
        ki_offset += 1
    kj_offset = 0
    for no, lo, mo in orbs[fj]:        
        if no == nj and lo == lj:
            break
        kj_offset += 1
    for (A,i,j,ki_base,kj_base), block_data in zip(block.samples, block.values):
        if A != cur_A:
            ham = dense[A]
            cur_A = A
        if block_type == 0:
            ham[ki_base+ki_offset:ki_base+ki_offset+2*li+1, kj_base+kj_offset:kj_base+kj_offset+2*lj+1] = block_data[:,0].reshape(2*li+1,2*lj+1)
        elif block_type == 2:
            ham[ki_base+ki_offset:ki_base+ki_offset+2*li+1, kj_base+kj_offset:kj_base+kj_offset+2*lj+1] = block_data[:,0].reshape(2*li+1,2*lj+1)
            ham[kj_base+kj_offset:kj_base+kj_offset+2*lj+1, ki_base+ki_offset:ki_base+ki_offset+2*li+1] = block_data[:,0].reshape(2*li+1,2*lj+1).T
        elif block_type == 1:
            ham[ki_base+ki_offset:ki_base+ki_offset+2*li+1, kj_base+kj_offset:kj_base+kj_offset+2*lj+1] += np.asarray(block_data[:,0].reshape(2*li+1,2*lj+1)  / np.sqrt(2), dtype=np.float64)
            ham[kj_base+ki_offset:kj_base+ki_offset+2*li+1, ki_base+kj_offset:ki_base+kj_offset+2*lj+1] += np.asarray(block_data[:,0].reshape(2*li+1,2*lj+1) / np.sqrt(2), dtype=np.float64)
        elif block_type == -1:
            ham[ki_base+ki_offset:ki_base+ki_offset+2*li+1, kj_base+kj_offset:kj_base+kj_offset+2*lj+1] += np.asarray(block_data[:,0].reshape(2*li+1,2*lj+1) / np.sqrt(2), dtype=np.float64)
            ham[kj_base+ki_offset:kj_base+ki_offset+2*li+1, ki_base+kj_offset:ki_base+kj_offset+2*lj+1] -= np.asarray(block_data[:,0].reshape(2*li+1,2*lj+1) / np.sqrt(2), dtype=np.float64)
        



In [133]:
np.linalg.norm(dense[10] - hams[10])

2.8031757299712627e-16

In [124]:
hams[0][14]

array([1.5284446523487805, -1.3608802251214152, -1.0105189297980637,
       -0.6902644277689021, -0.0178662805508044, -7.95851559664695e-17,
       -0.40085031489053513, -0.043435687415332436,
       -8.068584708727507e-17, -0.030292005681611906,
       -4.31198767481943e-17, 0.062406175805169384,
       -2.459992422474338e-17, -0.045262433044569894, -0.7229782533545261,
       -0.7471964897461008, 0.8773753635102233, -0.04048557118592741,
       -6.283581600800853e-17, -0.5021899580812228, -0.5868927205194019,
       -0.1388597149832118, 0.5208324783542648, -4.669393030960783e-17],
      dtype=object)

In [125]:
dense[0][14]

array([ 1.52844465e+00, -1.36088023e+00, -1.01051893e+00, -6.90264428e-01,
       -1.78662806e-02, -1.59170312e-16, -8.01700630e-01, -8.68713748e-02,
       -1.61371694e-16, -6.05840114e-02, -4.31198767e-17,  6.24061758e-02,
       -2.45999242e-17, -4.52624330e-02,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00])

In [40]:
bb = np.where(ham_et.sparse["block_type"]==-1)[0]

In [41]:
ham_et.sparse[bb[0]]

(-1, 1, 1, 0, 1, 1, 0)

In [43]:
bb[0]

58

In [45]:
ham_et.block(58).values

array([[[0.0]],

       [[0.0]],

       [[0.0]],

       [[0.0]],

       [[0.0]],

       [[0.0]],

       [[0.0]],

       [[0.0]],

       [[0.0]],

       [[0.0]],

       [[0.0]],

       [[0.0]],

       [[0.0]],

       [[0.0]],

       [[0.0]],

       [[0.0]],

       [[0.0]],

       [[0.0]],

       [[0.0]],

       [[0.0]],

       [[0.0]],

       [[0.0]],

       [[0.0]],

       [[0.0]],

       [[0.0]],

       [[0.0]],

       [[0.0]],

       [[0.0]],

       [[0.0]],

       [[0.0]],

       [[0.0]],

       [[0.0]],

       [[0.0]],

       [[0.0]],

       [[0.0]],

       [[0.0]],

       [[0.0]],

       [[0.0]],

       [[0.0]],

       [[0.0]],

       [[0.0]],

       [[0.0]],

       [[0.0]],

       [[0.0]],

       [[0.0]],

       [[0.0]],

       [[0.0]],

       [[0.0]],

       [[0.0]],

       [[0.0]],

       [[0.0]],

       [[0.0]],

       [[0.0]],

       [[0.0]],

       [[0.0]],

       [[0.0]],

       [[0.0]],

       [[0.0]],

       [[0.0]]

In [13]:
ham_et.block(0).values

array([[[-20.534034434156133]]], dtype=object)